In [ ]:
# 导入pandas与numpy工具包。
import pandas as pd
import numpy as np

# 创建特征列表。
column_names = [
    'Sample code number', 'Clump Thickness', 'Uniformity of Cell Size',
    'Uniformity of Cell Shape', 'Marginal Adhesion',
    'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
    'Normal Nucleoli', 'Mitoses', 'Class'
]

# 使用pandas.read_csv函数从互联网读取指定数据。
data = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data',
    names=column_names)

# 将?替换为标准缺失值表示。
data = data.replace(to_replace='?', value=np.nan)
# 丢弃带有缺失值的数据（只要有一个维度有缺失）。
data = data.dropna(how='any')

# 输出data的数据量和维度。
data.shape

In [ ]:
data.head()

数据集的简介见：[breast-cancer-wisconsin](https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names)

我们得知该数据集共有 $699$ 个样本，$11$ 个特征，其中一列特征表示检索的 id, $9$ 列与肿瘤相关的医学特征以及一列表示肿瘤类型的数值。`2` 表示 benign (良性), `4` 表示 malignant (恶性). 该数据集中有 $16$ 个缺失值，并使用 `?` 标出。

## 划分数据集

In [ ]:
# 使用sklearn.cross_valiation里的train_test_split模块用于分割数据。
from sklearn.model_selection import train_test_split

# 随机采样25%的数据用于测试，剩下的75%用于构建训练集合。
X_train, X_test, y_train, y_test = train_test_split(
    data[column_names[1:10]],
    data[column_names[10]],
    test_size=0.25,
    random_state=33)

In [ ]:
# 查验训练样本的数量和类别分布。
y_train.value_counts()

In [ ]:
# 查验测试样本的数量和类别分布。
y_test.value_counts()

## 数据集的标准化

In [ ]:
# 从sklearn.preprocessing里导入StandardScaler。
from sklearn.preprocessing import StandardScaler
# 从sklearn.linear_model里导入LogisticRegression与SGDClassifier。
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

# 标准化数据，保证每个维度的特征数据方差为1，均值为0。使得预测结果不会被某些维度过大的特征值而主导。
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

# 使用 sklearn 进行  Logistics Regression 学习

In [ ]:
# 初始化LogisticRegression与SGDClassifier。
lr = LogisticRegression()
sgdc = SGDClassifier(max_iter=10)

In [ ]:
%%timeit
# 调用LogisticRegression中的fit函数/模块用来训练模型参数。
lr.fit(X_train, y_train)
# 使用训练好的模型lr对X_test进行预测，结果储存在变量lr_y_predict中。
lr_y_predict = lr.predict(X_test)

In [ ]:
%%timeit
# 调用SGDClassifier中的fit函数/模块用来训练模型参数。
sgdc.fit(X_train, y_train)
# 使用训练好的模型sgdc对X_test进行预测，结果储存在变量sgdc_y_predict中。
sgdc_y_predict = sgdc.predict(X_test)

In [ ]:
# 从sklearn.metrics里导入classification_report模块。
from sklearn.metrics import classification_report

# 使用逻辑斯蒂回归模型自带的评分函数score获得模型在测试集上的准确性结果。
print('Accuracy of LR Classifier:', lr.score(X_test, y_test))
# 利用classification_report模块获得LogisticRegression其他三个指标的结果。
print(
    classification_report(
        y_test, lr_y_predict, target_names=['Benign', 'Malignant']))

In [ ]:
# 使用随机梯度下降模型自带的评分函数score获得模型在测试集上的准确性结果。
print('Accuarcy of SGD Classifier:', sgdc.score(X_test, y_test))
# 利用classification_report模块获得SGDClassifier其他三个指标的结果。
print(
    classification_report(
        y_test, sgdc_y_predict, target_names=['Benign', 'Malignant']))

一般说来，`SGDClassifier` 训练速度快于 `LogisticRegression`，适用于 $10$ 万量级的数据；而 `LogisticRegression` 的参数是采用精确解析的方式,模型的精度会相对高一点, 但是对于 $10$ 万量级的数据内存开销很大，有点不切实际。

 ---------------------

# 使用 MXNet 实现 Logistics Regression

使用 SGD 进行优化

In [ ]:
import random
import mxnet as mx
from mxnet import autograd, nd


def data_iter(batch_size, features, labels, trainable=True):
    '''
    数据迭代器
    '''
    num_examples = len(features)
    indices = np.arange(num_examples)
    if trainable:
        random.shuffle(indices)  # 样本的读取顺序是随机的。
    for i in range(0, num_examples, batch_size):
        j = np.array(indices[i: min(i + batch_size, num_examples)])
        # take 函数根据索引返回对应元素。
        yield features.take(j, axis=0), labels.take(j, axis=0)


def LR(X, w, b):
    '''
    Logistics Regression
    '''
    z = nd.dot(X, w) + b
    exp = 1 / (1 + z.exp())
    return exp


def squared_loss(y_hat, y):
    '''
    损失函数
    '''
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2


def sgd(params, lr, batch_size):
    '''
    优化算法
    '''
    for param in params:
        param[:] = param - lr * param.grad / batch_size

In [ ]:
num_features = X_train.shape[1]
w = nd.random.normal(shape=(num_features, 1))+0.001
b = nd.zeros(1)
w.attach_grad()
b.attach_grad()

In [ ]:
lr = 0.1
num_epochs = 100
batch_size = 64

for epoch in range(num_epochs):  # 训练模型一共需要 num_epochs 个迭代周期。
    # 在一个迭代周期中，使用训练数据集中所有样本一次（假设样本数能够被批量大小整除）。
    # X 和 y 分别是小批量样本的特征和标签。
    for X, y in data_iter(32, X_train, y_train):
        X = nd.array(X)/255
        y = nd.array(y)
        with autograd.record():
            y_hat = LR(X, w, b)
            L = squared_loss(y_hat, y)
        L.backward()
        sgd([w, b], lr, batch_size)  # 使用小批量随机梯度下降迭代模型参数。
    train_l = squared_loss(LR(X, w, b), y)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().asnumpy()))

In [ ]:
LR_y_predict = LR(nd.array(X_test), w, b).asnumpy()
LR_y_predict = np.where(LR_y_predict > .5, 2, 4)

In [ ]:
# 从sklearn.metrics里导入classification_report模块。
from sklearn.metrics import classification_report

print(
    classification_report(
        y_test<3, LR_y_predict<3, target_names=['Benign', 'Malignant']))

In [ ]:
y_hat = nd.array([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = nd.array([0, 1])
nd.pick(y_hat, y)

In [ ]:
y_hat